Desafio 23: Tratamento de Dados de Vendas

📂 Dataset: Vendas de produtos com campos inconsistentes.
Colunas: Produto, Categoria, Loja, Mês, Quantidade_Vendida, Valor_Unitario

🎯 Tarefas de Tratamento:

    Corrija problemas de capitalização e espaços extras nas colunas Produto e Categoria.

    Padronize as categorias e os nomes dos produtos para ficarem consistentes.

    Converta as colunas Quantidade_Vendida e Valor_Unitario para formato numérico, tratando os valores inválidos como "", "N/A", NaN".

    Crie uma nova coluna Faturamento_Total multiplicando a quantidade pelo valor unitário.

    Verifique se há valores nulos no faturamento e justifique o motivo.

    Faça um resumo com o faturamento total por loja e por produto.

In [39]:
import pandas as pd

In [17]:
df = pd.read_csv("dia_23_vendas_tratamento.csv")
df.head()

,Produto,Categoria,Loja,Mês,Quantidade_Vendida,Valor_Unitario
0,Smartphone,Periféricos,Loja C,Março,27.0,3686.73
1,Smartphone,perifericos,Loja B,Janeiro,37.0,1666.33
2,Teclado,Eletrônicos,Loja A,Março,4.0,4090.82
3,Monitor,perifericos,Loja C,Janeiro,50.0,3585.71
4,Teclado,Eletrônicos,Loja B,Abril,86.0,4696.94


In [41]:
#Corrija problemas de capitalização e espaços extras nas colunas Produto e Categoria.
df["Produto"] = df["Produto"].str.title().str.strip()
df["Categoria"] = df["Categoria"].str.title().str.strip()

In [55]:
#Padronize as categorias e os nomes dos produtos para ficarem consistentes.
df["Categoria"] = df["Categoria"].replace({"Eletronicos": "Eletrônicos", "Perifericos" : "Periféricos"})
df.loc[df["Produto"].isin(["Notebook", "Smartphone", "Monitor"]), "Categoria"] = "Eletrônicos"
df.loc[df["Produto"].isin(["Mouse", "Teclado", "Fone De Ouvido"]), "Categoria"] = "Periféricos"
print(df[["Produto", "Categoria"]].value_counts())

Produto         Categoria  
Notebook        Eletrônicos    23
Mouse           Periféricos    22
Smartphone      Eletrônicos    21
Teclado         Periféricos    20
Monitor         Eletrônicos    17
Fone De Ouvido  Periféricos    17
Name: count, dtype: int64


In [59]:
#Converta as colunas Quantidade_Vendida e Valor_Unitario para formato numérico, tratando os valores inválidos como "", "N/A", NaN".
df["Quantidade_Vendida"] = pd.to_numeric(df["Quantidade_Vendida"], errors="coerce")
df["Valor_Unitario"] = pd.to_numeric(df["Valor_Unitario"], errors="coerce")

df["Quantidade_Vendida"] = df["Quantidade_Vendida"].fillna(
    df.groupby(["Produto", "Categoria"])["Quantidade_Vendida"].transform("mean")
)
df["Valor_Unitario"] = df["Valor_Unitario"].fillna(
    df.groupby(["Produto", "Categoria"])["Valor_Unitario"].transform("mean")
)

In [61]:
#Crie uma nova coluna Faturamento_Total multiplicando a quantidade pelo valor unitário.
df["Faturamento_Total"] = df["Quantidade_Vendida"] * df["Valor_Unitario"]
df.head()

,Produto,Categoria,Loja,Mês,Quantidade_Vendida,Valor_Unitario,Faturamento_Total
0,Smartphone,Eletrônicos,Loja C,Março,27.0,3686.73,99541.71
1,Smartphone,Eletrônicos,Loja B,Janeiro,37.0,1666.33,61654.21
2,Teclado,Periféricos,Loja A,Março,4.0,4090.82,16363.28
3,Monitor,Eletrônicos,Loja C,Janeiro,50.0,3585.71,179285.50
4,Teclado,Periféricos,Loja B,Abril,86.0,4696.94,403936.84


In [69]:
#Verifique se há valores nulos no faturamento e justifique o motivo.
print(f"Quantidade de valores Nulos em 'Faturamento_Total': {df["Faturamento_Total"].isnull().sum()}")
print("Não há valores nulos pois todos os dados foram tratados")

Quantidade de valores Nulos em 'Faturamento_Total': 0
Não há valores nulos pois todos os dados foram tratados


In [83]:
#Faça um resumo com o faturamento total por loja e por produto.
pd.options.display.float_format = '{:,.2f}'.format
resumoPorLoja = df.groupby("Loja")["Faturamento_Total"].sum().reset_index().sort_values(by="Faturamento_Total", ascending=False)
print(f"Resumo do Faturamento Total por Loja: \n\n {resumoPorLoja}\n")

resumoPorProduto = df.groupby("Produto")["Faturamento_Total"].sum().reset_index().sort_values(by="Faturamento_Total", ascending=False)
print(f"Resumo do Faturamento Total por Produto: \n\n{resumoPorProduto}")

Resumo do Faturamento Total por Loja: 

      Loja  Faturamento_Total
1  Loja B       5,524,050.25
0  Loja A       4,865,579.91
2  Loja C       4,284,796.57

Resumo do Faturamento Total por Produto: 

          Produto  Faturamento_Total
2           Mouse       3,136,762.20
5         Teclado       3,015,885.42
3        Notebook       2,838,387.98
4      Smartphone       2,231,653.16
1         Monitor       1,769,896.26
0  Fone De Ouvido       1,681,841.72
